<a href="https://colab.research.google.com/github/jialinli-21/Li_DSPN_S25/blob/main/Exercise15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 15: Power analyses

This  assignment is designed to give you practice with Monte Carlo methods to conduct power analyses via simulation. You won't need to load in any data for this homework. We will, however, be using parts of the homework from last week.

1. Simulating data 1/1
2. run_analysis function 2/2
3. repeat_analysis() function 3/3
4. Testing different sample sizes 2/2
5. Reflection 2/2

---
## 1. Simulating data (1 points)


Pull your `simulate_data()` function from your last homework and add it below.

As a reminder, this function simulates the relationship between age, word reading experience, and reading comprehension skill.

`c` is reading comprehension, and `x` is word reading experience.

In [ ]:
sample_size = 100 # How many children in data set?
age_lo = 80     # minimum age, in months
age_hi = 200    # maximum age, in months
beta_xa = 0.5   # amount by which experience changes for increase of one month in age
beta_x0 = -5    # amount of experience when age = 0 (not interpretable, since minimum age for this data is 80 months)
sd_x = 50       # standard dev of gaussian noise term, epsilon_x
beta_ca = 0.8   # amount that comprehension score improves for every increase of one unit in age
beta_cx = 3     # amount that comprehension score improves for every increase of one unit in reading experience
beta_c0 = 10    # comprehension score when reading experience is 0.
sd_c = 85      # standard dev of gaussian noise term, epsilon_c

simulate_data <- function(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
      # generate a
      age = runif(sample_size, min = age_lo, max = age_hi)
      epsilon_x = rnorm(sample_size, mean = 0,sd=sd_x)
      x = beta_xa*age + beta_x0 + epsilon_x

      # generate c
      epsilon_c = rnorm(sample_size, mean = 0, sd=sd_c)
      c = beta_ca*age + beta_cx*x + beta_c0 + epsilon_c

      return(data.frame(age=age,x=x,c=c))
}

dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
head(dat)

,age,x,c
,<dbl>,<dbl>,<dbl>
1,177.56928,66.873213,215.3935
2,88.42791,55.810222,228.6128
3,107.93609,46.020403,304.3204
4,183.93809,64.301820,265.0491
5,102.31577,91.354184,290.3989
6,93.18194,9.295217,148.7705


---
## 2. `run_analysis()` function (2 pts)

Last week, we looked at whether word reading experience(`x`) mediated the relation between `age` and reading comprehension (`c`).

Now we're going to use our `simulate_data()` function to conduct a power analysis. The goal is to determine how many participants we would need in order to detect both the mediated and the direct effects in this data.

*Note: We're going to pretend for the sake of simplicity that we don't have any control over the ages of the children we get (so ages are generated using `runif(sample_size, age_lo, age_hi)`, although of course this would be an unusual situation in reality.*

First, write a function, `run_analysis()`, that takes in simulated data, runs **your mediation from last week**, and returns a vector containing the ACME and ADE estimates and p-values (these are the `d0`, `d0.p`, `z0`, and `z0.p` features of the mediated model object, e.g., `fitMed$d0.p`). Print this function's output for the data we simulated previously.

In [ ]:
# WRITE YOUR CODE HERE
install.packages("mediation")
library(mediation)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘rbibutils’, ‘checkmate’, ‘Rdpack’, ‘zoo’, ‘gridExtra’, ‘htmlTable’, ‘viridis’, ‘Formula’, ‘minqa’, ‘nloptr’, ‘reformulas’, ‘RcppEigen’, ‘mvtnorm’, ‘sandwich’, ‘lpSolve’, ‘Hmisc’, ‘lme4’


Loading required package: MASS

Loading required package: Matrix

Loading required package: mvtnorm

Loading required package: sandwich

mediation: Causal Mediation Analysis
Version: 4.5.0




In [ ]:
run_analysis <- function(dat){
  fitM <- lm(x ~ age,     data=dat)
  fitY <- lm(c ~ x + age, data=dat)
  fitMed <- mediate(fitM, fitY, treat="age", mediator="x")

    acme <- fitMed$d0
    acme_p <- fitMed$d0.p
    ade <- fitMed$z0
    ade_p <- fitMed$z0.p

  return(c(ACME = acme, ACME_p = acme_p, ADE = ade, ADE_p = ade_p))

}

results <- run_analysis(dat)
print(results)


    ACME   ACME_p      ADE    ADE_p 
1.838976 0.000000 1.258226 0.000000 


---
## 3. `repeat_analysis()` function (3 pts)

Next fill in the function `repeat_analysis()` below so that it simulates and analyzes data `num_simulations` times. Store the outputs from each simulation in the `simouts` matrix. Calculate and return the coverage across all the simulations run for both ACME and ADE.

In [ ]:
repeat_analysis <- function(num_simulations, alpha, sample_size, age_lo, age_hi,
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
    # Initialize simouts matrix for storing each output from run_analysis()
    simouts <- matrix(rep(NA, num_simulations*4), nrow=num_simulations, ncol=4)

    # Start simulating
    for (i in 1:num_simulations) {
      dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
      simouts[i,] <- run_analysis(dat)

    }

    # Calculate coverage for both ACME and ADE estimates using p-values in simouts
    ACME_cov = mean(simouts[,2] <= alpha)
    ADE_cov =  mean(simouts[,4] <= alpha)

    return(list(ACME_cov = ACME_cov, ADE_cov = ADE_cov))
}

Now run the `repeat_analysis()` function using the same parameter settings as above, for 10 simulations, with an alpha criterion of 0.01.

In [ ]:
result_rep <- repeat_analysis(num_simulations = 10, alpha = .01, sample_size, age_lo, age_hi,
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)

print(result_rep)



$ACME_cov
[1] 0.8

$ADE_cov
[1] 0.7



---
## 4. Testing different sample sizes (2 pts)

Finally, do the same thing (10 simulations, alpha criterion of 0.01) but for 5 different sample sizes: 50, 75, 100, 125, 150. You can do this using `map` (as in the tutorial), or a simple `for` loop, or by calculating each individually. Up to you! This should take around 3 minutes to run.

In [ ]:
results_50 <- repeat_analysis(num_simulations = 10, alpha = .01, sample_size = 50, age_lo, age_hi,
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)

results_75 <- repeat_analysis(num_simulations = 10, alpha = .01, sample_size = 75, age_lo, age_hi,
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)

results_100 <- repeat_analysis(num_simulations = 10, alpha = .01, sample_size = 100, age_lo, age_hi,
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)

results_125 <- repeat_analysis(num_simulations = 10, alpha = .01, sample_size = 125, age_lo, age_hi,
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)

results_150 <- repeat_analysis(num_simulations = 10, alpha = .01, sample_size = 150, age_lo, age_hi,
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)


Print your results.

In [ ]:
df <- rbind(results_50,results_75,results_100,results_125,results_150)
print(df)

            ACME_cov ADE_cov
results_50  0.3      0.3    
results_75  0.6      0.5    
results_100 0.8      0.8    
results_125 0.9      0.9    
results_150 1        0.6    


## 5. Reflection (2 pts)

If this were a real power analysis, we'd want to run more simulations per sample size (to get a more precise estimate of power) and we may also want to test out some other values of the parameters we used to simulate our data. However, what would you conclude just based on the results above?

> *The coverages increased when the sample size increased at the beginning, however, it started to decrease when the sample size reached 150.*
>

**Given** how we generated the data, why was the direct effect harder to detect than the mediated effect?
> *We generated the relationship between the mediator (x) and the predictor (age) first before we generated the relationship among the outcome (c), the mediator and the predictor. This order connected the mediator and the predictor stronger.*

**DUE:** 5pm EST, April 3, 2024

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *Someone's Name*